In [1]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.tools.yfinance import YFinanceTools
from phi.tools.duckduckgo import DuckDuckGo

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [4]:
#Creating Agent-1
Research_agent_1=Agent(
    name= "Companies research Agent",
    role="Search the web for the information",
    model=Groq(id = "llama-3.3-70b-versatile"),
    tools=[DuckDuckGo()],
    instructions=["Give in detailed information","Always include sources"],
    show_tool_calls=True,
    markdown=True
)

In [9]:
#example working of Agent-1
company="Forethought Learning"
response = Research_agent_1.run(company)

# Only extracts the 'content'
research_text = response.content

print(research_text)

c:\Users\Sai_K\Desktop\Multi-Agentic-AI\.venv\Lib\site-packages\phi\tools\duckduckgo.py:65: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(


### Forethought Learning Overview
Forethought Learning is an organization that focuses on developing and providing educational resources to enhance learning outcomes. To find more detailed information about Forethought Learning, I will need to search the web.


Running:
 - duckduckgo_search(max_results=5, query=Forethought Learning)

Forethought Learning is an organization that provides students with hands-on work experience and mentorship under top professionals. They have various programs and projects that help students build their profiles and gain valuable experience. According to their website and social media profiles, Forethought Learning has worked with over 50,000 students and offers 1-on-1 mentorship with top 1% of professors and professionals. 

Sources:
- [Forethought Learning](https://www.forethought.co.in/)
- [Forethought Learning | LinkedIn](https://in.linkedin.com/company/forethoughtlearning)
- [Forethought Learning (@forethoughtlearning) - Instagram](https://www.instag

In [10]:
#case genaration Agent
Case_genaration_Agent_2=Agent(
    name="Use case genarator Agent",
    role="Analyze the given industry/company research and generate AI/ML/GenAI based use cases to improve operations, customer experience, and efficiency.",
    model=Groq(id="llama-3.3-70b-versatile"),
    instructions=[
        "Take the input research carefully.",
        "Generate at least 5 detailed use cases.",
        "Ensure each use case mentions how GenAI, ML or LLMs can be applied.",
        "Suggest creative as well as industry-standard ideas."
    ],
    show_tool_calls=True,
    markdown=True
    
)

In [11]:
#example execution of Agent-2
use_case_response = Case_genaration_Agent_2.run(research_text)
use_case_response_text=use_case_response.content
print(use_case_response_text) 

### Introduction to Forethought Learning Use Cases
Forethought Learning is an innovative organization that provides hands-on work experience and mentorship to students under top professionals. To further enhance their operations, customer experience, and efficiency, the following AI/ML/GenAI-based use cases can be explored:

### Use Case 1: **Personalized Learning Paths**
* **Description:** Utilize Machine Learning (ML) algorithms to analyze student performance, interests, and career goals to create personalized learning paths.
* **Application:** Implement a recommendation system that suggests relevant programs, projects, and mentorship opportunities to students based on their profiles.
* **Benefits:** Improved student engagement, enhanced learning outcomes, and increased mentorship effectiveness.

### Use Case 2: **AI-Powered Mentorship Matching**
* **Description:** Develop a GenAI-powered matching system to connect students with suitable mentors based on their expertise, interests, a

In [12]:
#Convert Agent-2's verbose output into a search query.
"""Example Input: 
        "**Use Case 1**: AI-powered chatbot for Workday (leveraging NLP)..."
    Output: 
        "Workday chatbot NLP"
    """
import re

def extract_keywords(markdown_text: str) -> str:
    
    # Step 1: Remove markdown formatting
    clean_text = re.sub(r'\*+|`+|#+', '', markdown_text)
    
    # Step 2: Extract capitalized/noun phrases (crude but effective)
    keywords = re.findall(r'\b[A-Z][a-z]+(?:\s+[A-Z][a-z]+)*\b|\b\w{4,}\b', clean_text)
    
    # Step 3: Filter and limit keywords
    stopwords = {'for', 'and', 'the', 'with', 'using'} 
    keywords = [kw for kw in keywords if kw.lower() not in stopwords]
    
    return " ".join(keywords)

#Example running:
#agent2_output = "**Use Case**: AI-powered Workday chatbot (NLP, LLMs)..."
print(extract_keywords(use_case_response_text))  # Output: "Workday chatbot NLP LLMs"

Introduction Forethought Learning Use Cases
Forethought Learning innovative organization that provides hands work experience mentorship students under professionals To further enhance their operations customer experience efficiency following GenAI based cases explored Use Case Personalized Learning Paths
 Description Utilize Machine Learning algorithms analyze student performance interests career goals create personalized learning paths Application Implement recommendation system that suggests relevant programs projects mentorship opportunities students based their profiles Benefits Improved student engagement enhanced learning outcomes increased mentorship effectiveness Use Case Powered Mentorship Matching
 Description Develop GenAI powered matching system connect students suitable mentors based their expertise interests availability Application Use Natural Language Processing analyze mentor profiles student requirements program descriptions suggest optimal mentor mentee pairs Benefit

Using kaggle, hf, and github's api for data collection

In [13]:
from typing import List, Union
import json
# Ensure the output is a string (required for tool responses).

def format_output(results: Union[List[str], str]) -> str:
    if isinstance(results, list):
        return json.dumps(results)  # Convert list to JSON string
    return str(results)

In [14]:
from kaggle.api.kaggle_api_extended import KaggleApi
import os

def setup_kaggle():
    """Ensure kaggle.json exists or use environment variables."""
    if not os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
        raise FileNotFoundError("Place kaggle.json in ~/.kaggle/ or set KAGGLE_USERNAME/KAGGLE_KEY env vars.")

# Kaggle
def search_kaggle_datasets(query: str, max_results: int = 3) -> str:  # Return string!
    try:
        api = KaggleApi()
        api.authenticate()
        datasets = api.dataset_list(search=query)[:max_results]
        links = [f"https://www.kaggle.com/datasets/{d.ref}" for d in datasets]
        return format_output(links)  # Ensure string output
    except Exception as e:
        return format_output(f"Kaggle Error: {e}")

In [15]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()  # No errors = success!

In [16]:
Dataset_Collector_Agent_3 = Agent(
    name="Dataset Collector Agent",
    model=Groq(id="llama-3.3-70b-versatile"),
    tools=[search_kaggle_datasets,],
    instructions=[
        "You MUST search for datasets using tools: search_kaggle_datasets",
        "Collate the top 10 results from platform into a single list.",
        "For each link, strictly follow this format: '[PLATFORM: Dataset Title](URL)'.",
        "Example: '[Kaggle: Zomato Restaurant Data](https://www.kaggle.com/datasets/shrutimehta/zomato-restaurants-data)'",
        "If a platform yields no results, state that clearly. For example: 'No relevant datasets found on kaggle.'",
        "Never hallucinate links!"
    ],
    markdown=True,
)

In [17]:
import re

def extract_use_case_titles(markdown_text: str) -> list[str]:
    """Extract ### headings from markdown (use case titles)"""
    return re.findall(r'### \d+\. \*\*(.*?)\*\*', markdown_text)

# Example:
agent2_output = """
## Use Cases
### 1. **Intelligent Inventory Management**
### 2. **AI-Powered Chatbots**
"""
titles = extract_use_case_titles(use_case_response_text) 
print(titles)  # ['Intelligent Inventory Management', 'AI-Powered Chatbots']

[]


In [18]:
def create_search_query(title: str) -> str:
    """Convert use case title to dataset search query"""
    # Keep only nouns/verbs (remove connectors)
    keywords = re.sub(r'\b(for|using|with|the)\b', '', title)
    return f"{keywords} dataset"

# Example:
create_search_query("Intelligent Inventory Management") 
# -> "Intelligent Inventory Management dataset"

'Intelligent Inventory Management dataset'

In [19]:
import time
def find_datasets_for_use_cases(use_case_titles: list[str]) -> dict:
    results = {}
    for title in use_case_titles:
        query = create_search_query(title)
        print(f"Searching for datasets for: {title}")
        
        # --- Retry Logic with Exponential Backoff ---
        max_retries = 3
        base_delay = 2  # seconds
        
        for attempt in range(max_retries):
            try:
                # This is the API call that might fail
                response = Dataset_Collector_Agent_3.run(query)
                results[title] = response.content
                
                # If successful, break out of the retry loop
                break
                
            except Exception as e: # Ideally, catch the specific RateLimitError
                if "429" in str(e): # Check if the error is a rate limit error
                    if attempt < max_retries - 1:
                        # Calculate wait time: 2s, 4s, 8s, etc.
                        wait_time = base_delay * (2 ** attempt)
                        print(f"Rate limit hit for '{title}'. Retrying in {wait_time} seconds...")
                        time.sleep(wait_time)
                    else:
                        print(f"Failed to fetch datasets for '{title}' after {max_retries} attempts.")
                        results[title] = "Could not fetch datasets due to repeated rate limiting."
                else:
                    # Handle other types of errors
                    print(f"An unexpected error occurred for '{title}': {e}")
                    results[title] = "An unexpected error occurred."
                    break # Stop trying if it's not a rate limit issue
        
        # A small, polite delay between different use case searches
        time.sleep(1)
            
    return results

In [20]:
def generate_ai_use_cases_with_datasets(company: str) -> str:
    # Step 1: Run Agent 1 → Agent 2
    research = Research_agent_1.run(f"Find details about {company}").content
    use_cases_markdown = Case_genaration_Agent_2.run(research).content
    
    # Step 2: Process Agent 2's output
    use_case_titles = extract_use_case_titles(use_cases_markdown)
    datasets_by_use_case = find_datasets_for_use_cases(use_case_titles)
    
    # Step 3: Generate structured output
    output = ["## AI Use Cases with Recommended Datasets"]
    for title in use_case_titles:
        output.append(f"### {title}")
        output.append(datasets_by_use_case.get(title, "No relevant datasets found"))
    
    return "\n".join(output)

In [21]:
# Test with Walmart
result = generate_ai_use_cases_with_datasets(company)
print(result)

c:\Users\Sai_K\Desktop\Multi-Agentic-AI\.venv\Lib\site-packages\phi\tools\duckduckgo.py:65: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  ddgs = DDGS(


Searching for datasets for: Personalized Program Recommendations
Searching for datasets for: Intelligent Chatbot for Student Support
Searching for datasets for: Predictive Analytics for Student Success
Searching for datasets for: Automated Content Generation for Social Media
Searching for datasets for: Career Guidance and Job Matching
## AI Use Cases with Recommended Datasets
### Personalized Program Recommendations
Based on the search results, here are the top 10 datasets related to "Personalized Program Recommendations dataset" on Kaggle:
* [Kaggle: Customer Lifetime Value Analytics Case Study](https://www.kaggle.com/datasets/bhanupratapbiswas/customer-lifetime-value-analytics-case-study)
* [Kaggle: FitLife User Profiles](https://www.kaggle.com/datasets/gauravduttakiit/fitlife-user-profiles)
* [Kaggle: Courses](https://www.kaggle.com/datasets/kararhaitham/courses)
* [Kaggle: Online Course User Engagement Data](https://www.kaggle.com/datasets/thedevastator/online-course-user-engagemen